In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [2]:

# Function to create the dataset with timesteps
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data)-time_step-1):
        a = data[i:(i+time_step), :]
        X.append(a)
        Y.append(data[i + time_step, :])
    return np.array(X), np.array(Y)

# Load your dataset
file_name = 'waterTank_Golden_reduced.csv'
data = pd.read_csv(file_name)
data = data.values

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# Split the data into train and test sets
training_size = int(len(data) * 0.67)
test_size = len(data) - training_size
train_data, test_data = data[0:training_size, :], data[training_size:len(data), :]

# Create the datasets for training and testing
time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, data.shape[1])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(data.shape[1]))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform to get actual values
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform(y_train)
y_test = scaler.inverse_transform(y_test)

# Calculate evaluation metrics
def calculate_metrics(true, predicted):
    rmse = np.sqrt(mean_squared_error(true, predicted))
    mape = mean_absolute_percentage_error(true, predicted)
    r2 = r2_score(true, predicted)
    return rmse, mape, r2

train_rmse, train_mape, train_r2 = calculate_metrics(y_train, train_predict)
test_rmse, test_mape, test_r2 = calculate_metrics(y_test, test_predict)

print(f"Train RMSE: {train_rmse}, Train MAPE: {train_mape}, Train R2: {train_r2}")
print(f"Test RMSE: {test_rmse}, Test MAPE: {test_mape}, Test R2: {test_r2}")


Epoch 1/100
157/157 [==============================] - 26s 63ms/step - loss: 0.0347
Epoch 2/100
157/157 [==============================] - 10s 66ms/step - loss: 0.0087
Epoch 3/100
157/157 [==============================] - 12s 74ms/step - loss: 0.0052
Epoch 4/100
157/157 [==============================] - 11s 68ms/step - loss: 0.0041
Epoch 5/100
157/157 [==============================] - 10s 66ms/step - loss: 0.0036
Epoch 6/100
157/157 [==============================] - 11s 71ms/step - loss: 0.0033
Epoch 7/100
157/157 [==============================] - 11s 68ms/step - loss: 0.0030
Epoch 8/100
157/157 [==============================] - 11s 69ms/step - loss: 0.0028
Epoch 9/100
157/157 [==============================] - 11s 68ms/step - loss: 0.0025
Epoch 10/100
157/157 [==============================] - 11s 67ms/step - loss: 0.0024
Epoch 11/100
157/157 [==============================] - 12s 75ms/step - loss: 0.0025
Epoch 12/100
157/157 [==============================] - 11s 67ms/step - lo